# Spectral Clustering  

In this notebook, we will use cuGraph to identify the cluster in a test graph using Spectral Clustering with both the (A) Balance Cut metric, and (B) the Modularity Maximization metric


Notebook Credits
* Original Authors: Bradley Rees and James Wyles
* Last Edit: 05/03/2019

RAPIDS Versions: 0.7.0

Test Hardware
* GP100 32G, CUDA 9,2


## Introduction

Spectral clustering uses the eigenvectors of a Laplacian of the input graph to find a given number of clusters which satisfy a given quality metric. Balanced Cut and Modularity Maximization are such quality metrics. 

@See:  https://en.wikipedia.org/wiki/Spectral_clustering

To perform spectral clustering using the balanced cut metric in cugraph use:

__cugraph.spectralBalancedCutClustering(G, num_clusters, num_eigen_vects)__
<br>or<br>
__cugraph.spectralModularityMaximizationClustering(G, num_clusters, num_eigen_vects)__



Input
* __G__: A cugraph.Graph object
* __num_clusters__: The number of clusters to find
* __num_eig__: (optional) The number of eigenvectors to use

Returns
* __df__: cudf.DataFrame with two names columns:
    * df["vertex"]: The vertex id.
    * df["cluster"]: The assigned partition.


## cuGraph 0.7 Notice 
cuGraph version 0.7 has some limitations:
* Only Int32 Vertex ID are supported
* Only float (FP32) edge data is supported
* Vertex numbering is assumed to start at zero

These limitations are being addressed and will be fixed future versions.  
These example notebooks will illustrate how to manipulate the data so that it comforms to the current limitations 


----

### Test Data
We will be using the Zachary Karate club dataset 
*W. W. Zachary, An information flow model for conflict and fission in small groups, Journal of
Anthropological Research 33, 452-473 (1977).*


![Karate Club](./img/zachary_black_lines.png)

Zachary used a min-cut flow model to partition the graph into two clusters, shown by the circles and squares.  Zarchary wanted just two cluster based on a conflict that caused the Karate club to break into two separate clubs.  Many social network clustering methods identify more that two social groups in the data.

In [ ]:
# Import needed libraries
import cugraph
import cudf
import numpy as np
from collections import OrderedDict

### Read the CSV datafile using cuDF

In [ ]:
# Test file  
datafile='./data/karate-data.csv'

# Read the data file
cols = ["src", "dst"]

dtypes = OrderedDict([
        ("src", "int32"), 
        ("dst", "int32")
        ])

gdf = cudf.read_csv(datafile, names=cols, delimiter='\t', dtype=list(dtypes.values()) )

### Adjusting the vertex ID
Let's adjust all the vertex IDs to be zero based.  We are going to do this by adding two new columns with the adjusted IDs

In [ ]:
gdf["src"] = gdf["src"] - 1
gdf["dst"] = gdf["dst"] - 1

In [ ]:
# The algorithm requires that there are edge weights.  In this case all the weights are being ste to 1
gdf["data"] = cudf.Series(np.ones(len(gdf), dtype=np.float32))

In [ ]:
# Look at the first few data records - the output should be two colums src and dst
gdf.head().to_pandas()

In [ ]:
# verify data type
gdf.dtypes

Everything looks good, we can now create a graph

In [ ]:
# create a CuGraph 
G = cugraph.Graph()
G.add_edge_list(gdf["src"], gdf["dst"], gdf["data"])

----
#### Define and print function, but adjust vertex ID so that they match the illustration

In [ ]:
def print_cluster(_df, id):
    
    _f = _df.query('cluster == @id')
  
    part = []
    for i in range(len(_f)):
        part.append(_f['vertex'][i] + 1)
    print(part)

----
#### Using Balanced Cut

In [ ]:
# Call spectralBalancedCutClustering on the graph for 3 clusters
# using 3 eigenvectors:
bc_gdf = cugraph.spectralBalancedCutClustering(G, 3, num_eigen_vects=3)

In [ ]:
# Check the edge cut score for the produced clustering
score = cugraph.analyzeClustering_edge_cut(G, 3, bc_gdf['cluster'])
score

In [ ]:
# See which nodes are in cluster 0:
print_cluster(bc_gdf, 0)

In [ ]:
# See which nodes are in cluster 1:
print_cluster(bc_gdf, 1)

In [ ]:
# See which nodes are in cluster 2:
print_cluster(bc_gdf, 2)

----
#### Modularity Maximization
Let's now look at the clustering using the modularity maximization metric

In [ ]:
# Call spectralModularityMaximizationClustering on the graph for 3 clusters
# using 3 eigenvectors:
mm_gdf = cugraph.spectralModularityMaximizationClustering(G, 3, num_eigen_vects=3)

In [ ]:
# Check the modularity score for the produced clustering
score = cugraph.analyzeClustering_modularity(G, 3, mm_gdf['cluster'])
score

In [ ]:
# See which nodes are in cluster 0:
print_cluster(mm_gdf, 0)

In [ ]:
print_cluster(mm_gdf, 1)

In [ ]:
print_cluster(mm_gdf, 2)

Notice that the two metrics produce different results

___
Copyright (c) 2019, NVIDIA CORPORATION.

Licensed under the Apache License, Version 2.0 (the "License");  you may not use this file except in compliance with the License. You may obtain a copy of the License at http://www.apache.org/licenses/LICENSE-2.0

Unless required by applicable law or agreed to in writing, software distributed under the License is distributed on an "AS IS" BASIS, WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied. See the License for the specific language governing permissions and limitations under the License.
___